---
title: Part 2. Introduction to Actinia
description: Learn how to use actinia to create a cloud-based processing environment for GRASS GIS.
format: 
  html:
    toc: true
    code-tools: true
    code-copy: true
    code-fold: false
author:
    - Corey T. White
    - Vaclav Petras
date: '2024-09-10'
keep-ipynb: true
toc: true
toc-depth: 4
image:  images/webinar_title.webp
categories: [geospatial, GRASS v8.5, jupyter, lidar, STAC]
page-layout: full
title-block-banner: true
---

In [58]:
!figlet "actinia 101"

            _   _       _         _  ___  _ 
  __ _  ___| |_(_)_ __ (_) __ _  / |/ _ \/ |
 / _` |/ __| __| | '_ \| |/ _` | | | | | | |
| (_| | (__| |_| | | | | | (_| | | | |_| | |
 \__,_|\___|\__|_|_| |_|_|\__,_| |_|\___/|_|
                                            


## A REST API for GRASS GIS

- [GRASS GIS](https://grass.osgeo.org/) is a Geospatial Processing Engine
- Open Source (GPL v2)
- Developed by International and Multi-institutional groups and individuals (GRASS Development Team)
- Member of the Open Source Geospatial Foundatispace
- Recieved Open Source Security Foundation (OpenSSF) Best Practices Badge - 2024

```{mermaid}
flowchart LR
  A[Client] -->|Http Request| B[Actinia REST API]
  B <--> D[GRASS GIS Processing Node]
  B -->|Http Response| A
```


## Import Python Libraries

In [2]:
import os
import subprocess
from pprint import pprint
import sys
import json
import time

import requests
from requests.auth import HTTPBasicAuth

## Setup Environment

Deploy Docker container with Actinia

Follow Instructions at : [https://github.com/OpenPlainsInc/foss4gna-2024-workshop](https://github.com/OpenPlainsInc/foss4gna-2024-workshop)

### Setup Variables

In [ ]:
ACTINIA_USER = 'actinia-gdi'
ACTINIA_PASSWORD = 'actinia-gdi'
ACTINIA_VERSION = 'v3'
ACTINIA_BASEURL = 'http://localhost:8088'
ACTINIA_URL = ACTINIA_BASEURL + "/api/" + ACTINIA_VERSION

ACTINIA_AUTH = HTTPBasicAuth(ACTINIA_USER, ACTINIA_PASSWORD)

In [4]:
def print_as_json(data):
    print(json.dumps(data, indent=2))

## actinia REST API

[actinia](https://github.com/actinia-org)
[Docker Image](https://github.com/actinia-org/actinia-docker/tree/main)

### Tools

- [actinia-python-client](https://github.com/actinia-org/actinia-python-client)
- [actinia_openapi_python_client](https://github.com/OpenPlainsInc/actinia_openapi_python_client)

## Acinia Basics

In [7]:
url = f"{ACTINIA_URL}/version"
response = requests.get( url)
print_as_json(response.json().get('version'))

"4.14.1"


## User Roles

1. **Superadmin**
    - Create, modify and delete users
    - Read/write access to all databases
    - Can access all API calls

2. **Admin**
    - All API Calls
    - Create, modify and delete users with the maximum user-role user of the same user grou
    - Access to persistent databases that were granted by a **superadmin**

3. **User**
    - Can run computational tasks in ephemeral and user specific databases
    - Create, modify and delete locations in a user specific database
    - Create, modify and delete mapsets in user specific databases
    - Limited access to API calls
    - Has limited access to persistent databases

4. **Guest**
    - Has very limited access to API calls
    - Limited access to persistent databases

:::aside
https://actinia-org.github.io/actinia-core/actinia_concepts/
:::


### GRASS Projects

Let's start by requesting a list of GRASS projects from actinia.

```{mermaid}
flowchart LR
  A["/api/v3/locations"]
```

In [ ]:
url = f"{ACTINIA_URL}/locations"
response = requests.get(url, auth=ACTINIA_AUTH)
print_as_json(response.json().get('locations'))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Now we will get extra information about a specific project.

```{mermaid}
flowchart LR
  A["/api/v3/locations/"]
  A --- C["{location_id}"]
  C --- D["/info"]
```

In [11]:
location_id = 'nc_spm_08'
url = f"{ACTINIA_URL}/locations/{location_id}/info"

response = requests.get( url, auth=ACTINIA_AUTH)

pprint(response.json().get('process_results'))

None


### Mapsets

We will now get a list of mapsets for a specific project.

```{mermaid}
flowchart LR
  A["/api/v3/locations/"]
  A --- B("{location_id}")
  B --- C["/mapsets"]
```

In [8]:
# mapset = 'nc_spm_08'
location_id = 'nc_spm_08_grass7'
url = f"{ACTINIA_URL}/locations/{location_id}/mapsets"

response = requests.get( url, auth=ACTINIA_AUTH)

pprint(response.json().get('process_results'))

['PERMANENT']


Now let's get extra information about a specific mapset `PERMANENT`.

```{mermaid}
flowchart LR
  A["/api/v3/locations/"]
  A --- B("{location_id}")
  B --- C["/mapsets/"]
  C --- D("{mapset_id}")
  D --- E["/info"]
```

In [10]:
mapset_id = 'PERMANENT'
location_id = 'nc_spm_08_grass7'
url = f"{ACTINIA_URL}/locations/{location_id}/mapsets/{mapset_id}/info"

response = requests.get( url, auth=ACTINIA_AUTH)

pprint(response.json().get('process_results'))

{'projection': 'PROJCRS["NAD83(HARN) / North '
               'Carolina",BASEGEOGCRS["NAD83(HARN)",DATUM["NAD83 (High '
               'Accuracy Reference Network)",ELLIPSOID["GRS '
               '1980",6378137,298.257222101,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4152]],CONVERSION["SPCS83 '
               'North Carolina zone (meter)",METHOD["Lambert Conic Conformal '
               '(2SP)",ID["EPSG",9802]],PARAMETER["Latitude of false '
               'origin",33.75,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8821]],PARAMETER["Longitude '
               'of false '
               'origin",-79,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8822]],PARAMETER["Latitude '
               'of 1st standard '
               'parallel",36.1666666666667,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8823]],PARAMETER["Latitude '
               'of 2nd standard '
               'parallel",34.3333333333333,ANGLEUNIT["degree",0.017453

### Layers

#### Raster Layers

Let's see all of the mapsets raster layers.

```{mermaid}
flowchart LR
  A["/api/v3/locations/"]
  A --- B("{location_id}")
  B --- C["/mapsets/"]
  C --- D("{mapset_id}")
  D --- E["/raster_layers"]
```

In [12]:
mapset_id = 'PERMANENT'
location_id = 'nc_spm_08_grass7'
url = f"{ACTINIA_URL}/locations/{location_id}/mapsets/{mapset_id}/raster_layers"

response = requests.get( url, auth=ACTINIA_AUTH)

pprint(response.json().get('process_results'))

['aspect',
 'basin_50K',
 'boundary_county_500m',
 'cfactorbare_1m',
 'cfactorgrow_1m',
 'el_D782_6m',
 'el_D783_6m',
 'el_D792_6m',
 'el_D793_6m',
 'elev_lid792_1m',
 'elev_ned_30m',
 'elev_srtm_30m',
 'elev_state_500m',
 'elevation',
 'elevation_shade',
 'facility',
 'geology_30m',
 'lakes',
 'landclass96',
 'landcover_1m',
 'landuse96_28m',
 'lsat7_2002_10',
 'lsat7_2002_20',
 'lsat7_2002_30',
 'lsat7_2002_40',
 'lsat7_2002_50',
 'lsat7_2002_61',
 'lsat7_2002_62',
 'lsat7_2002_70',
 'lsat7_2002_80',
 'ncmask_500m',
 'ortho_2001_t792_1m',
 'roadsmajor',
 'slope',
 'soilsID',
 'soils_Kfactor',
 'streams_derived',
 'towns',
 'urban',
 'zipcodes',
 'zipcodes_dbl']


#### Vector Layers

```{mermaid}
flowchart LR
  A["/api/v3/locations/"]
  A --- B("{location_id}")
  B --- C["/mapsets/"]
  C --- D("{mapset_id}")
  D --- E["/vector_layers"]
```

In [20]:
mapset_id = 'PERMANENT'
location_id = 'nc_spm_08_grass7'
url = f"{ACTINIA_URL}/locations/{location_id}/mapsets/{mapset_id}/vector_layers"

payload = {}
headers = {}

response = requests.get( url, auth=ACTINIA_AUTH, headers=headers)

pprint(response.json().get('process_results'))

['P079214',
 'P079215',
 'P079218',
 'P079219',
 'boundary_county',
 'boundary_municp',
 'bridges',
 'busroute1',
 'busroute11',
 'busroute6',
 'busroute_a',
 'busroutesall',
 'busstopsall',
 'census_wake2000',
 'censusblk_swwake',
 'comm_colleges',
 'elev_lid792_bepts',
 'elev_lid792_cont1m',
 'elev_lid792_randpts',
 'elev_lidrural_mrpts',
 'elev_lidrural_mrptsft',
 'elev_ned10m_cont10m',
 'firestations',
 'geodetic_pts',
 'geodetic_swwake_pts',
 'geology',
 'geonames_NC',
 'geonames_wake',
 'hospitals',
 'lakes',
 'nc_state',
 'overpasses',
 'poi_names_wake',
 'precip_30ynormals',
 'precip_30ynormals_3d',
 'railroads',
 'roadsmajor',
 'schools_wake',
 'soils_general',
 'soils_wake',
 'streams',
 'streets_wake',
 'swwake_10m',
 'urbanarea',
 'usgsgages',
 'zipcodes_wake']


### Processing

#### Syncronous Processing

Emphemeral process

```{mermaid}
flowchart LR
  A["/api/v3/locations/"]
  A --- B("{location_id}")
  B --- C["/mapsets/"]
  C --- D("{mapset_id}")
  D --- E["/processing"]
```

In [21]:
mapset_id = 'PERMANENT'
location_id = 'nc_spm_08_grass7'
url = f"{ACTINIA_URL}/locations/{location_id}/mapsets/{mapset_id}/processing"

data = {}

response = requests.post( url, auth=ACTINIA_AUTH, json=data)

pprint(response.json().get('process_results'))

None


#### Asyncronous Processing

```{mermaid}
flowchart LR
  A["/api/v3/locations/"]
  A --- B("{location_id}")
  B --- C["/mapsets/"]
  C --- D("{mapset_id}")
  D --- E["/processing_async"]
```

In [22]:
mapset_id = 'PERMANENT'
location_id = 'nc_spm_08_grass7'
url = f"{ACTINIA_URL}/locations/{location_id}/mapsets/{mapset_id}/processing_async"

data = {}

response = requests.post( url, auth=ACTINIA_AUTH, json=data)

pprint(response.json().get('process_results'))

{}


## Let's look at the complete API within `PostMan` or `Insomnia`.

- [Postman](https://www.postman.com/)
- [Insomnia](https://insomnia.rest/download)

```
https://localhost:8080/api/v3/swagger-ui/index.html
```

### Process Chains

Let move on to creating process chains in [Part 3](./process_chains.ipynb).